# Import libs

In [1]:
import torch
from timm.models import create_model
from musk import utils, modeling
from PIL import Image
from transformers import XLMRobertaTokenizer
from timm.data.constants import IMAGENET_INCEPTION_MEAN, IMAGENET_INCEPTION_STD
import torchvision
from huggingface_hub import login
login(<HF Token>)
device = torch.device("cuda:2")


/home/xiangjx/anaconda3/envs/musk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extract Image Embeddings

- Set `ms_aug = True` for:  
  - Linear probe classification  
  - Multiple Instance Learning  

- Set `ms_aug = False` for:  
  - Zero-shot tasks (e.g., image-image retrieval and image-text retrieval)


In [2]:
# >>>>>>>>>>>> load model >>>>>>>>>>>> #
model_config = "musk_large_patch16_384"
model = create_model(model_config).eval()
utils.load_model_and_may_interpolate("hf_hub:xiangjx/musk", model, 'model|module', '')
model.to(device, dtype=torch.float16)
model.eval()
# <<<<<<<<<<<< load model <<<<<<<<<<<< #

# >>>>>>>>>>>> process image >>>>>>>>>>> #
# load an image and process it
img_size = 384
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(img_size, interpolation=3, antialias=True),
    torchvision.transforms.CenterCrop((img_size, img_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=IMAGENET_INCEPTION_MEAN, std=IMAGENET_INCEPTION_STD)
])

img = Image.open('assets/lungaca1014.jpeg').convert("RGB")  # input image
img_tensor = transform(img).unsqueeze(0)
with torch.inference_mode():
    image_embeddings = model(
        image=img_tensor.to(device, dtype=torch.float16),
        with_head=False, # We only use the retrieval head for image-text retrieval tasks.
        out_norm=True,
        ms_aug=True  # by default it is False, `image_embeddings` will be 1024-dim; if True, it will be 2048-dim.
        )[0]  # return (vision_cls, text_cls)

print(image_embeddings.shape)

Load ckpt from hf_hub:xiangjx/musk
torch.Size([1, 2048])


# Multimodal Retrieval Example

In [3]:
# >>>>>>>>>>>> load model >>>>>>>>>>>> #
model_config = "musk_large_patch16_384"
model = create_model(model_config).eval()
utils.load_model_and_may_interpolate("hf_hub:xiangjx/musk", model, 'model|module', '')
model.to(device, dtype=torch.float16)
model.eval()
# <<<<<<<<<<<< load model <<<<<<<<<<<< #

# >>>>>>>>>>>> process image >>>>>>>>>>> #
# load an image and process it
img_size = 384
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(img_size, interpolation=3, antialias=True),
    torchvision.transforms.CenterCrop((img_size, img_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=IMAGENET_INCEPTION_MEAN, std=IMAGENET_INCEPTION_STD)
])

img = Image.open('assets/lungaca1014.jpeg').convert("RGB")  # input image
img_tensor = transform(img).unsqueeze(0)
with torch.inference_mode():
    image_embeddings = model(
        image=img_tensor.to(device, dtype=torch.float16),
        with_head=True,  # We only use the retrieval head for image-text retrieval tasks.
        out_norm=True
        )[0]  # return (vision_cls, text_cls)
# <<<<<<<<<<< process image <<<<<<<<<<< #

# >>>>>>>>>>> process language >>>>>>>>> #
# load tokenzier for language input
tokenizer = XLMRobertaTokenizer("./musk/models/tokenizer.spm")
labels = ["lung adenocarcinoma",
            "benign lung tissue",
            "lung squamous cell carcinoma"]

texts = ['histopathology image of ' + item for item in labels]
text_ids = []
paddings = []
for txt in texts:
    txt_ids, pad = utils.xlm_tokenizer(txt, tokenizer, max_len=100)
    text_ids.append(torch.tensor(txt_ids).unsqueeze(0))
    paddings.append(torch.tensor(pad).unsqueeze(0))

text_ids = torch.cat(text_ids)
paddings = torch.cat(paddings)
with torch.inference_mode():
    text_embeddings = model(
        text_description=text_ids.to(device),
        padding_mask=paddings.to(device),
        with_head=True, 
        out_norm=True
    )[1]  # return (vision_cls, text_cls)
# <<<<<<<<<<<< process language <<<<<<<<<<< #

# >>>>>>>>>>>>> calculate similarity >>>>>>> #
with torch.inference_mode():
    # expected prob:[0.3782, 0.3247, 0.2969]  --> lung adenocarcinoma
    sim = model.logit_scale * image_embeddings @ text_embeddings.T
    prob = sim.softmax(dim=-1)
    print(prob)

Load ckpt from hf_hub:xiangjx/musk
tensor([[0.3782, 0.3247, 0.2969]], device='cuda:2', dtype=torch.float16)
